1. Merging the complete DF

In [ ]:
import glob
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt

all_dataframes =    [ ]
all_dataframes.append(pd.read_csv('soil_analysis.csv'))
all_dataframes.append(pd.read_csv('leaf_analysis.csv'))
all_dataframes.append(pd.read_csv('petiole_analysis.csv'))

2. Rename column's name

In [ ]:
for df in all_dataframes:
        print(df.columns.values)

i = 1
for df in all_dataframes:
        m = {}
        for name in df.columns.values[6:]:
                m[name] = str(name+str(i))
        i += 1
        all_dataframes[i-2] = df.rename(columns=m)

for df in all_dataframes:
        print(df.columns.values)

df = all_dataframes[0]
df = pd.merge(df, all_dataframes[1], how='outer', on=['ID Company', 'ID Lot', 'Nation', 'Province', 'Place', 'Year'])
df = pd.merge(df, all_dataframes[2], how='outer', on=['ID Company', 'ID Lot', 'Nation', 'Province', 'Place', 'Year'])

df.to_csv('complete_dataset.csv')



3. Remove records having values of only 1 phase out of the three (soil/leaf/petiole)

In [ ]:
features = df.columns.values

features_soil = features[6:30]
features_leaf = features[30:41]
features_petiole = features[41:]

def check_empty(sample, features):
        for feature in features:
                if not math.isnan(sample[feature]):
                        return 0
        return 1

to_drop = []
for index, row in df.iterrows():
        if check_empty(row, features_soil) + check_empty(row, features_leaf) + check_empty(row, features_petiole) >= 2:
                to_drop.append(index)
df = df.drop(to_drop)


3.1 Integration height values

In [ ]:
ita_h = pd.read_csv("additional_datasets/italy_heights.csv")
slo_h = pd.read_csv("additional_datasets/slovenia_heights.csv")
cro_h = pd.read_csv("additional_datasets/croatia_heights.csv")
spa_h = pd.read_csv("additional_datasets/spagna_heights.csv")


ita_h['HEIGHT'] = ita_h['HEIGHT'].str.replace(',', '.').astype(float)

heights = []
for index, row in df.iterrows():
        if row["Nation"] == "Italia":
                i = ita_h.loc[ita_h["NAME"] == row["Place"]].index.values[0]
                heights.append(ita_h.at[i, "HEIGHT"])
        elif row["Nation"] == "Slovenia":
                i = slo_h.loc[slo_h["NAME"] == row["Place"]].index.values[0]
                heights.append(slo_h.at[i, "HEIGHT"])
        elif row["Nation"] == "Croazia":
                i = cro_h.loc[cro_h["NAME"] == row["Place"]].index.values[0]
                heights.append(cro_h.at[i, "HEIGHT"])
        elif row["Nation"] == "Spagna":
                i = spa_h.loc[spa_h["NAME"] == row["Place"]].index.values[0]
                heights.append(spa_h.at[i, "HEIGHT"])
        else:
                print("Error: Nation not allowed")


if "Height" not in df.columns.values.tolist():
        df.insert(6, "Height", heights, True)


3.2 Integration temperature and humidity

In [ ]:
temp_hum = pd.read_csv("temperature&humidity/complete_temp_hum.csv")
temp_hum = temp_hum.drop(temp_hum.columns.values[0], axis = 1)

phase2 = temp_hum.loc[temp_hum["Phase"] == 2].copy()
phase2.rename(columns = {"T avg [˚C]": "T avg 2", "T min [˚C]": "T min 2", "T max [˚C]": "T max 2", "Humidity": "Humidity2"}, inplace = True)
phase2 = phase2.drop("Phase", axis = 1)

phase3 = temp_hum.loc[temp_hum["Phase"] == 3].copy()
phase3.rename(columns = {"T avg [˚C]": "T avg 3", "T min [˚C]": "T min 3", "T max [˚C]": "T max 3", "Humidity": "Humidity3"}, inplace = True)
phase3 = phase3.drop("Phase", axis = 1)

temp_hum = pd.merge(phase2, phase3, how = 'outer', on = ["Province", "Year"])

df = pd.merge(df, temp_hum, how = "left", on = ["Province", "Year"])
df.to_csv('complete_dataset.csv')



3.3 Integration coordinates

In [ ]:
coo = pd.read_csv("geo_coords.csv", sep = "\t")
coo = coo.rename({"City" : "Place"}, axis = 1)
df = pd.merge(df, coo, how="left", on="Place")

df.to_csv('complete_dataset.csv')

4. Data cleaning: fix the '<x' and '>x' data

In [ ]:
feature = "Cu3"
less = []            #elements presenting '< x'
great = []          #elements presenting '> x'
normal = []        #legit elements (Nan are not considered here)

for val in df[feature]:
        if str(val)[0] == '<':
                temp = str(val)[1:]
                temp = float(temp.replace(',', '.'))
                if temp not in less:
                        less.append(temp)
                        print("less than "+str(less[-1]))
        elif str(val)[0] == '>':
                temp = str(val)[1:]
                temp = float(temp.replace(',', '.'))
                if temp not in great:
                        great.append(temp)
                        print("greater than "+str(great[-1]))
        elif not math.isnan(float(str(val).replace(',', '.'))):
                temp = float(str(val).replace(',', '.'))
                normal.append(temp)

plt.plot(normal)
print("MIN = "+str(min(normal)))
print("MAX = "+str(max(normal)))

features = df.columns.values[7:]
types = df.dtypes[7:]

print("Max values of the affected columns")

i = 0
for feature in features:
        #only columns with dtype=object need to be fixed
        if types[i] != "object":
                i += 1
                continue
        new_col = []
        
        #find the max value
        m = 0
        for val in df[feature]:
                if str(val)[0] == '<':
                        continue
                elif str(val)[0] == '>':
                        temp = temp = str(val)[1:]
                        temp = float(temp.replace(',', '.'))
                        m = max(m, temp)
                else:
                        temp = float(str(val).replace(',', '.'))
                        m = max(m, temp)
        print(feature+"        "+str(m))
        
        #compose the new/clean column of values
        for val in df[feature]:
                temp = val
                if str(val)[0] == '<':
                        temp = 0.0
                elif str(val)[0] == '>':
                        temp = m
                else:
                        temp = float(str(val).replace(',', '.'))
                new_col.append(temp)
        
        #replace old column with the clean one
        df[feature] = new_col
        i += 1

df.to_csv('complete_dataset.csv')


#Handling sparse missing values
#Let's count first the number of missing values in the current version of the dataset

features = df.columns.values
features_soil = features[7:31]
features_leaf = features[31:42]
features_petiole = features[42:53]


def count_missing(record, features, miss):
        c = 0
        for feature in features:
                if math.isnan(record[feature]):
                        miss[feature] += 1
                        c += 1
        return c

miss = {}
for feature in features[7:53]:
        miss[feature] = 0

for index, row in df.iterrows():
        if not check_empty(row, features_soil):
                count_missing(row, features_soil, miss)
        if not check_empty(row, features_leaf):
                count_missing(row, features_leaf, miss)
        if not check_empty(row, features_petiole):
                count_missing(row, features_petiole, miss)

print("Total number of rows = " + str(df.shape[0]))

plt.figure(figsize=(20,6))
plt.bar(miss.keys(), list(miss.values()), align='center')
plt.xticks(rotation=90)
#plt.xticks(range(len(miss)), list(miss.keys()))
plt.show

p1 = df.filter(items = features_soil, axis = 1)
for index, row in p1.iterrows():
        if check_empty(row, p1.columns.values):
                p1 = p1.drop(index)

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=4)
p1 = imputer.fit_transform(p1)
p1 = pd.DataFrame.from_records(p1, columns = features_soil)


p1_original = df.filter(items = features_soil, axis = 1)
p1_final = []

i = 0
for index, row in p1_original.iterrows():
        if check_empty(row, p1_original.columns.values):
                p1_final.append(row)
        else:
                p1_final.append(p1.iloc[i].copy())
                i += 1
p1_final = pd.DataFrame.from_records(p1_final)

for c in p1_final.columns:
        df[c] = p1_final[c]
def count_missing(record, features, miss):
        c = 0
        for feature in features:
                if math.isnan(record[feature]):
                        miss[feature] += 1
                        c += 1
        return c

miss = {}
for feature in features[7:53]:
        miss[feature] = 0

for index, row in df.iterrows():
        if not check_empty(row, features_soil):
                count_missing(row, features_soil, miss)
        if not check_empty(row, features_leaf):
                count_missing(row, features_leaf, miss)
        if not check_empty(row, features_petiole):
                count_missing(row, features_petiole, miss)

print("Total number of rows = " + str(df.shape[0]))

plt.figure(figsize=(20,6))
plt.bar(miss.keys(), list(miss.values()), align='center')
plt.xticks(rotation=90)
#plt.xticks(range(len(miss)), list(miss.keys()))
plt.show

df.to_csv('complete_dataset.csv')


